In [2]:
import sqlite3
import config as cfg
import pandas as pd

In [3]:
from tqdm.notebook import tqdm

tqdm.pandas()

# reviews data

In [ ]:
with cfg.PATHS.RAW_REVIEWS.open() as f:
    df = pd.read_json(f, lines=True)

df.head()

In [ ]:
df = df[["review_id", "user_id", "business_id", "stars", "text", "useful", "funny", "cool", "date"]]

In [ ]:
# convert date field to timedelta
df["date"] = pd.to_datetime(df["date"])
df["review_age"] = df["date"].progress_apply(lambda x: (cfg.DATE_COLLECTED - x).days)
df.drop(columns=["date"], inplace=True)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df["review_stars"] = df["stars"]
df.drop(columns=["stars"], inplace=True)

In [ ]:
df.to_sql("reviews", con=sqlite3.connect(cfg.PATHS.DB), if_exists="replace", index=False)

In [ ]:
del df

# business data

In [ ]:
with cfg.PATHS.RAW_BUSINESS.open() as f:
    df = pd.read_json(f, lines=True)

df.head()

In [ ]:
df = df[["business_id", "city", "stars", "categories"]]

In [ ]:
df["categories_list_string"] = df["categories"].progress_apply(str)

In [ ]:
df.drop(columns=["categories"], inplace=True)

In [ ]:
df.to_sql("business", con=sqlite3.connect(cfg.PATHS.DB), if_exists="replace", index=False)

In [ ]:
del df

# user data

In [ ]:
with cfg.PATHS.RAW_USER.open() as f:
    df = pd.read_json(f, lines=True)

df.head()

In [ ]:
df = df[["user_id", "review_count", "yelping_since", "fans"]]

In [ ]:
df["account_age"] = (cfg.DATE_COLLECTED - pd.to_datetime(df["yelping_since"])).dt.days

In [ ]:
df.drop(columns=["yelping_since"], inplace=True)

In [ ]:
df.to_sql("users", con=sqlite3.connect(cfg.PATHS.DB), if_exists="replace", index=False)

In [ ]:
del df

# join reviews to business and user data

In [4]:
df = pd.read_sql("SELECT * FROM reviews JOIN business ON reviews.business_id = business.business_id JOIN users ON reviews.user_id = users.user_id", con=sqlite3.connect(cfg.PATHS.DB))

In [5]:
df = df.sample(frac=cfg.proportion, random_state=42)

In [6]:
cfg.proportion

1.0

In [7]:
df.drop(columns=["business_id", "user_id", "review_id"], inplace=True)

In [8]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df = df[df["text"] != ""]

In [9]:
df.head()

,text,useful,funny,cool,review_age,review_stars,city,stars,categories_list_string,review_count,fans,account_age
5527377,A cheaper alternative to a full blown AYCE hot...,17,3,7,1163,4,Philadelphia,4.0,"Taiwanese, Hot Pot, Restaurants, Bubble Tea, Food",140,6,3602
389000,We liked this place so much I ate there again ...,1,0,1,1299,5,Tucson,4.0,"Restaurants, Salad, Pakistani, Indian, Cocktai...",31,0,4094
556758,"Pretty limited menu, but good authentic food. ...",2,1,1,1577,5,Philadelphia,4.0,"Halal, Restaurants, Mexican, Steakhouses",21,0,3902
4074689,I have a 2014 Ford Escape and a very strange t...,0,0,0,2480,4,Tampa,3.5,"Automotive, Transmission Repair, Car Dealers, ...",1,0,2480
3687671,I've been a member of CrossFit En Fuego for ne...,3,1,1,3015,5,Land O Lakes,5.0,"Interval Training Gyms, Trainers, Fitness & In...",145,7,3791


# output results as pickled dataframe

In [10]:
df.to_pickle(cfg.PATHS.CLEAN_DATA_PICKLE)

In [11]:
# only keep values where sum of useful, funny, and cool is greater than 10
df = df[df["useful"] + df["funny"] + df["cool"] <= 10]

In [12]:
df.to_pickle(cfg.PATHS.OBJECTS_FOLDER / "clean_data_unpopular.pickle")

In [13]:
df.shape

(6767070, 12)